In [1]:
from ortools.linear_solver import pywraplp
import openpyxl
import os
import sys
import time
import datetime
from datetime import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import pandas as pd

from random import seed
from random import random

In [83]:
solver = pywraplp.Solver.CreateSolver('GLOP')
solver.EnableOutput()
# solver.SetNumThreads(10)

In [2]:
path = './advanced/data/'

In [3]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

DEM = pd.read_excel(path+'dataLarge.xlsx',sheet_name='DEM')
CAPA = pd.read_excel(path+'dataLarge.xlsx',sheet_name='CAPA')
PCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='PCOST')
SCOST = pd.read_excel(path+'dataLarge.xlsx',sheet_name='SCOST')
ISTOCK = pd.read_excel(path+'dataLarge.xlsx',sheet_name='ISTOCK')
ITEM = DEM['ITEM'].drop_duplicates().to_list()
RES = CAPA['RES'].drop_duplicates().to_list()
TIME = DEM['TIME'].drop_duplicates().to_list()

endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Load Data :" + str(endDateTime-startDateTime))

Start : 2023-Jan-31 17:51:15.010227
End : 2023-Jan-31 17:51:18.398070
Load Data :0:00:03.387843


In [65]:
arr_DEM = DEM.to_numpy()
arr_CAPA = CAPA.to_numpy()
arr_PCOST = PCOST.to_numpy()
arr_SCOST = SCOST.to_numpy()
arr_ISTOCK = ISTOCK.to_numpy()

In [66]:
adj_DEM = np.zeros((len(ITEM),len(TIME)))
adj_CAPA = np.zeros((len(RES),len(TIME)))
adj_PCOST = np.zeros((len(ITEM),len(RES)))
adj_SCOST = np.zeros((len(ITEM)))
adj_ISTOCK = np.zeros((len(ITEM)))

In [76]:
for i in range(len(arr_DEM)):
    demand = arr_DEM[i][2]
    np.put(adj_DEM,i,demand)
    
for i in range(len(arr_CAPA)):
    capacity = arr_CAPA[i][2]
    np.put(adj_CAPA,i,capacity)
    
for i in range(len(arr_PCOST)):
    productcost = arr_PCOST[i][2]
    np.put(adj_PCOST,i,productcost)
    
for i in range(len(arr_SCOST)):
    stockcost = arr_SCOST[i][1]
    np.put(adj_SCOST,i,stockcost)
    
for i in range(len(arr_ISTOCK)):
    initstock = arr_ISTOCK[i][1]
    np.put(adj_ISTOCK,i,initstock)

In [88]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

prodqty = {}
storeqty = {}

for i in range(len(ITEM)):
    prodqty[i]=[[solver.NumVar(0, solver.infinity(), 'prodqty[%i][%i][%i]' %(i,j,k)) for k in range(len(TIME))] for j in range(len(RES))]

for i in range(len(ITEM)):
    storeqty[i] = [solver.NumVar(0, solver.infinity(), 'storeqty[%i][%i]' %(i,k)) for k in range(len(TIME))]
        
print('Number of variables =', solver.NumVariables())

Start : 2023-Jan-31 21:40:45.110895
Number of variables = 1010000


In [89]:
for i in range(len(ITEM)) :
    for k in range(len(TIME)):
        if k == 0 :
            solver.Add(solver.Sum(prodqty[i][j][k] for j in range(len(RES))) == int(adj_DEM[i][k]) - int(adj_ISTOCK[i]))
        else :
            solver.Add(solver.Sum(prodqty[i][j][k] for j in range(len(RES))) + storeqty[i][k-1] - storeqty[i][k] == int(adj_DEM[i][k]))

In [90]:
for j in range(len(RES)):
    for k in range(len(TIME)):
        solver.Add(solver.Sum(prodqty[i][j][k] for i in range(len(ITEM))) <= int(adj_CAPA[j][k]) )

In [91]:
print('Number of constraints =', solver.NumConstraints())

endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Make Constraints:" + str(endDateTime-startDateTime))

Number of constraints = 20000
End : 2023-Jan-31 21:40:48.014944
Make Constraints:0:00:02.904049


In [92]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

obj_exp = 0
# obj = solver.Sum(((prodqty[i][j][k] * PCOST[(PCOST['ITEM']==i+1)&(PCOST['RES']==j+1)].reset_index()['PCOST'] for i in range(len(ITEM))) for j in range(len(RES))) for k in range(len(TIME)))
for i in range(len(ITEM)):
    for j in range(len(RES)):
        for k in range(len(TIME)):
            obj_exp = obj_exp + prodqty[i][j][k] * int(adj_PCOST[i][j]) + storeqty[i][k] * int(adj_SCOST[i])
            
endDateTime = dt.now()
print("End : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print("Make Objective:" + str(endDateTime-startDateTime))

Start : 2023-Jan-31 21:41:40.710090
End : 2023-Jan-31 21:41:47.382622
Make Objective:0:00:06.672532


In [94]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

solver.Minimize(obj_exp)
solver.Solve()

endDateTime = dt.now()
print("Solve Problem : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print(endDateTime-startDateTime)

Start : 2023-Jan-31 21:42:46.031418
Solve Problem : 2023-Jan-31 21:42:51.668233
0:00:05.636815


In [95]:
startDateTime = dt.now()
print("Start : " + startDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))

arr_prod = np.empty((0,4), int)
for i in range(len(ITEM)):
    for j in range(len(RES)):
        for k in range(len(TIME)):
           arr_prod = np.append(arr_prod,np.array([[i+1,j+1,k+1,prodqty[i][j][k].solution_value()]]), axis=0)

df_prod= pd.DataFrame(arr_prod,columns = ['ITEM','RES','TIME','PROD'])

arr_store = np.empty((0,3), int)
for i in range(len(ITEM)):
    for k in range(len(TIME)):
        arr_store = np.append(arr_store,np.array([[i+1,k+1,storeqty[i][k].solution_value()]]), axis=0)

df_store = pd.DataFrame(arr_store,columns = ['ITEM','TIME','STORE'])

df_prod.to_csv(path+'prodqty2.csv',index=False)
df_store.to_csv(path+'storeqty2.csv',index=False)

    
endDateTime = dt.now()
print("Write output : " + endDateTime.strftime("%Y-%b-%d %H:%M:%S.%f"))
print(endDateTime-startDateTime)

Start : 2023-Jan-31 21:42:51.683908
Write output : 2023-Jan-31 21:58:19.466673
0:15:27.782765
